In [ ]:
import re

import pandas as pd
import torch
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
import ast


In [ ]:
saq_ground_truth = pd.read_csv("../data/test_split_saq.csv")
mcq_ground_truth = pd.read_csv("../data/test_split_mcq.csv")

In [ ]:
model_results_path_saq = "../results/test/base_model_saq_prediction.tsv"

In [ ]:
saq_predictions = pd.read_csv(model_results_path_saq, sep='\t')
saq_predictions[saq_predictions["ID"]=="New-in-33"]

In [ ]:
saq_ground_truth["annotations"]

In [ ]:
def score_answer(model_answer, annotations):
    model_answer = model_answer.strip().lower()
    ann = ast.literal_eval(annotations)

    for entry in ann:
        for ans in entry["en_answers"]:
            if model_answer == ans.lower():
                return entry["count"]
    return 0

In [ ]:
merged = saq_ground_truth.merge(saq_predictions, on="ID", how="left")
merged["score"] = merged.apply(
    lambda row: score_answer(row["answer"], row["annotations"]) if pd.notna(row["annotations"]) else 0,
    axis=1
)
total_score = merged["score"].sum()
print(f"Total Score = {total_score}")
merged

In [ ]:
duplicates = saq_ground_truth[saq_ground_truth.duplicated(subset=["ID"], keep=False)]
duplicates.sort_values(by="ID")